# Блок 1. Введение > Введение

Объединение данных из разных источников. Почему это важно?
В реальных задачах, как правило, мы не имеем готовой к использованию таблицы, а получаем данные из разных источников, и очень часто приходится объединять несколько таблиц в одну. 

Например, при построении воронки продаж для сквозной аналитики, обогащении выгрузки данными другой системы, сравнении полноты данных в системах и при других задачах.

При этом требования к объемам данных постоянно растут. Если у вас нет под рукой настроенной базы данных или личного разработчика, то объединить даже «небольшие» таблицы в пару миллионов строк по визитам на сайт и покупкам в CRM станет проблемой, а по современным меркам это сравнительно небольшие таблицы. При этом объединять их надо сразу по нескольким измерениям. Например, чтобы получить конверсию в продажи на каждый день в разбивке по рекламному источнику и региону.

На этом занятии мы научимся эффективно решать такие задачи средствами Python, увидим, как несколько строк кода существенно экономят нам время. Для начала рассмотрим практические примеры решения таких задач и уточним технические особенности.

# Блок 2. Объединение таблиц по одному измерению > Шаг 1. Преобразование таблицы в словарь

### Описание кейса для объединения таблиц

На данном занятии мы будем использовать данные о покупках из файла, сгруппированных по источнику, каналу и дате. а также аналогичный файл по визитам на сайт, часть из которых привела к покупке.

Файл с визитами по источникам (visits_by_source.txt) 

Файл с покупками по источникам (orders_by_source.txt)

Наша задача — посчитать конверсию из визитов в покупки в разрезе источников трафика.

Т. е. для каждой пары значений source и medium нам нужно взять количество визитов и покупок и совместить их в одной таблице. Для тех, кто знаком с SQL, это аналог операции JOIN. На первом шаге нам понадобится самый простой случай объединения этих файлов по одному столбцу (источнику).

### visits_by_source.txt — в первом столбце стоит источник трафика, во втором — сумма визитов:

burgerclub 1197

city-magazine 528

facebook 3144
...

### orders_by_source.txt — в первом столбце стоит источник трафика, во втором — количество покупок, в третьем — суммарная стоимость покупок:

burgerclub 10 185

city-magazine 5 81

direct 5 88

Важное замечание: сейчас наши файлы очень малы, и объединить их легко можно даже в Excel. В реальной жизни такие файлы намного больше, поэтому мы представим, что решаем более сложную задачу со следующими условиями:

1. Файл visits_by_source.txt может быть любого размера (мы обрабатываем его построчно). Представим, что это выгрузка из базы данных на 50 Гб или поток данных в виде транзакций из базы данных.

То есть мы считаем файл условно «бесконечным».

2. Второй файл orders_by_source.txt помещается в оперативной памяти компьютера. Однако в памяти мы можем обрабатывать довольно большие файлы. Например, файл в 1 гигабайт свободно помещается в памяти даже не особо мощного ноутбука. 

То есть файл большой, но помещается в оперативную память.

### Функция поиска по файлу покупок
Наша задача будет состоять в следующем: нужно написать функцию, на вход которой будем подавать очередную строчку из файла visits_by_source.txt. В ответ должны получать количество покупок из файла orders_by_source.txt, которое соответствует этой строчке. Т. е. в самом простом случае в файле orders_by_source.txt надо найти строчку с таким же источником и вернуть значение из второго столбца.

Начнем писать такую функцию:

In [1]:
def searchForLine( source ):
    """
    Функция по названию источника source ищет соответствующую строку в файле orders_by_source.txt.
    Возвращает количество покупок, соответствующее источнику source. Если источник не найден, то возвращает 0    
    Пример
    searchForLine('burgerclub')
    10    
    searchForLine('source_123')
    0
    """

Для получения определенного значения по названию источника отлично подойдет словарь.

Можно было, например, поместить наши пары значений в набор листов и пробегать все его значения. Но это будет очень долго в случае больших файлов. К тому же если источника в файле покупок нет, то "проход" по этому листу будет впустую.

Словарь позволит гораздо быстрее найти в файле покупок нужный источник.

Давайте переведем наш файл с покупками orders_by_source.txt к словарю. Назовем его orders_dict и начнем писать цикл:

In [5]:
orders_dict = {}
with open('orders_by_source.txt', 'r') as f:
    for line in f:
        line = line.strip().split('\t') 
        print (line)

['burgerclub', '10', '185']
['city-magazine', '5', '81']
['direct', '5', '88']
['facebook', '5', '91']
['food-delivery', '10', '173']
['foody', '3', '66']
['google', '77', '1319']
['newsletter', '5', '98']
['promo', '68', '1181']
['vk', '2', '29']
['yandex', '104', '1818']


In [6]:
visits_dict = {}
with open('visits_by_source.txt', 'r') as fv:
    for line_v in fv:
        line_v = line_v.strip().split('\t') 
        print (line_v)

['burgerclub', '1197']
['city-magazine', '528']
['facebook', '3144']
['food-delivery', '1184']
['foody', '421']
['google', '10961']
['newsletter', '637']
['promo', '7405']
['vk', '256']
['yandex', '11757']
['direct', '2156']


### Упражнение

Дополните алгоритм, который записывает данные файла orders_by_source.txt в словарь orders_dict: в качестве ключей используйте первое значение листа line (обозначим источник как source), а в качестве значений - второе значение (orders_count). Обратите внимание, что число orders_count скорее всего считается из файла как строковая переменная. Используйте функцию int, чтобы получить численное значение.

Какое количество покупок выдаст строчка print(orders_dict['promo'])?

### Добавление и обновление ключей
Добавление новых пар в словарь происходит достаточно просто:

Добавляем ключ "туфля" со значением "род обуви, закрывающей ногу не выше щиколотки"

dictionary['туфля'] = 'род обуви, закрывающей ногу не выше щиколотки'
Обновление существующих значений происходит абсолютно также:

Обновляем ключ "туфля" и присваиваем ему значение "хорошая туфля"

dictionary['туфля'] = 'хорошая туфля'

Удаляем значение с ключом "противостоять" из словаря

del dictionary['противостоять']

Метод вернёт 0 в случае, если данного ключа не существует
story_count.get('два', 0)

In [9]:
orders_dict = {}
with open('orders_by_source.txt', 'r') as f:
    for line in f:
        line = line.strip().split('\t') 
        print (line)
        print (line[0])
        print (line[1])
        orders_dict[line[0]] = int(line[1])
        print(orders_dict)
print('ОТВЕТ: ', orders_dict['promo'])

['burgerclub', '10', '185']
burgerclub
10
{'burgerclub': 10}
['city-magazine', '5', '81']
city-magazine
5
{'burgerclub': 10, 'city-magazine': 5}
['direct', '5', '88']
direct
5
{'burgerclub': 10, 'city-magazine': 5, 'direct': 5}
['facebook', '5', '91']
facebook
5
{'burgerclub': 10, 'city-magazine': 5, 'direct': 5, 'facebook': 5}
['food-delivery', '10', '173']
food-delivery
10
{'burgerclub': 10, 'city-magazine': 5, 'direct': 5, 'facebook': 5, 'food-delivery': 10}
['foody', '3', '66']
foody
3
{'burgerclub': 10, 'city-magazine': 5, 'direct': 5, 'facebook': 5, 'food-delivery': 10, 'foody': 3}
['google', '77', '1319']
google
77
{'burgerclub': 10, 'city-magazine': 5, 'direct': 5, 'facebook': 5, 'food-delivery': 10, 'foody': 3, 'google': 77}
['newsletter', '5', '98']
newsletter
5
{'burgerclub': 10, 'city-magazine': 5, 'direct': 5, 'facebook': 5, 'food-delivery': 10, 'foody': 3, 'google': 77, 'newsletter': 5}
['promo', '68', '1181']
promo
68
{'burgerclub': 10, 'city-magazine': 5, 'direct': 5, '

In [10]:
print(orders_dict)

{'burgerclub': 10, 'city-magazine': 5, 'direct': 5, 'facebook': 5, 'food-delivery': 10, 'foody': 3, 'google': 77, 'newsletter': 5, 'promo': 68, 'vk': 2, 'yandex': 104}


In [11]:
visits_dict = {}
with open('visits_by_source.txt', 'r') as fv:
    for line_v in fv:
        line_v = line_v.strip().split('\t') 
#        print (line_v)
#        print (line_v)
#        print (line_v[0])
#        print (line_v[1])
        visits_dict[line_v[0]] = int(line_v[1])
        print(visits_dict)
#print('ОТВЕТ: ', visits_dict['burgerclub'])

{'burgerclub': 1197}
{'burgerclub': 1197, 'city-magazine': 528}
{'burgerclub': 1197, 'city-magazine': 528, 'facebook': 3144}
{'burgerclub': 1197, 'city-magazine': 528, 'facebook': 3144, 'food-delivery': 1184}
{'burgerclub': 1197, 'city-magazine': 528, 'facebook': 3144, 'food-delivery': 1184, 'foody': 421}
{'burgerclub': 1197, 'city-magazine': 528, 'facebook': 3144, 'food-delivery': 1184, 'foody': 421, 'google': 10961}
{'burgerclub': 1197, 'city-magazine': 528, 'facebook': 3144, 'food-delivery': 1184, 'foody': 421, 'google': 10961, 'newsletter': 637}
{'burgerclub': 1197, 'city-magazine': 528, 'facebook': 3144, 'food-delivery': 1184, 'foody': 421, 'google': 10961, 'newsletter': 637, 'promo': 7405}
{'burgerclub': 1197, 'city-magazine': 528, 'facebook': 3144, 'food-delivery': 1184, 'foody': 421, 'google': 10961, 'newsletter': 637, 'promo': 7405, 'vk': 256}
{'burgerclub': 1197, 'city-magazine': 528, 'facebook': 3144, 'food-delivery': 1184, 'foody': 421, 'google': 10961, 'newsletter': 637, '

In [12]:
print(visits_dict)

{'burgerclub': 1197, 'city-magazine': 528, 'facebook': 3144, 'food-delivery': 1184, 'foody': 421, 'google': 10961, 'newsletter': 637, 'promo': 7405, 'vk': 256, 'yandex': 11757, 'direct': 2156}


### Объединение таблиц

Теперь мы можем использовать наш словарь в функции поиска количества покупок. Для более удобного использования будем передавать словарь как аргумент функции searchForLine:

keys() — возвращает список ключей;

values() — возвращает список значений

for key, value in d.items():
        print(key, value)

In [15]:
def searchForLine( source, dict_s ):
    """
    searchForLine('burgerclub')
    1197   
    searchForLine('source_123')
    0
    """
    #print(dict_s)
    if source in dict_s:
        #print (source)
        return int( dict_s[source] )
    else:
        return (0)

Функция по названию источника source ищет соответствующую строку в файле orders_by_source.txt.

Возвращает количество покупок, соответствующее источнику source. Если источник не найден, то возвращает 0

source.get('два', 0)

In [13]:
print(orders_dict)

{'burgerclub': 10, 'city-magazine': 5, 'direct': 5, 'facebook': 5, 'food-delivery': 10, 'foody': 3, 'google': 77, 'newsletter': 5, 'promo': 68, 'vk': 2, 'yandex': 104}


In [16]:
# Проверим работает ли наша функция как мы ожидаем:
print (visits_dict)
print(searchForLine('burgerclub', visits_dict))

{'burgerclub': 1197, 'city-magazine': 528, 'facebook': 3144, 'food-delivery': 1184, 'foody': 421, 'google': 10961, 'newsletter': 637, 'promo': 7405, 'vk': 256, 'yandex': 11757, 'direct': 2156}
1197


In [17]:
print(searchForLine('source_123', visits_dict))

0


### Упражнение

Напишите цикл, в котором в каждую строчку файла visits_by_source.txt подставляется соответствующее количество покупок orders. Т. е. цикл заканчивается следующей строкой:

...

print(source, visits, orders, orders / visits)

Выведите на экран конверсию визитов в покупки для каждого источника (т. е. отношение orders к visits). Какое значение конверсии у источника vk? Ответ округлите до тысячных (т. е. ответ будет иметь формат 0.009).

In [30]:
for k, v in visits_dict.items():
    print (k, str(v))
    source = k
    print ('source = ', source)
    visits = v
    print ('visits = ', visits)
    orders = searchForLine(k, orders_dict)
    print ('orders = ', orders)
    print(source, visits, orders, orders / visits)

burgerclub 1197
source =  burgerclub
visits =  1197
orders =  10
burgerclub 1197 10 0.00835421888053467
city-magazine 528
source =  city-magazine
visits =  528
orders =  5
city-magazine 528 5 0.00946969696969697
facebook 3144
source =  facebook
visits =  3144
orders =  5
facebook 3144 5 0.0015903307888040711
food-delivery 1184
source =  food-delivery
visits =  1184
orders =  10
food-delivery 1184 10 0.008445945945945946
foody 421
source =  foody
visits =  421
orders =  3
foody 421 3 0.007125890736342043
google 10961
source =  google
visits =  10961
orders =  77
google 10961 77 0.007024906486634432
newsletter 637
source =  newsletter
visits =  637
orders =  5
newsletter 637 5 0.007849293563579277
promo 7405
source =  promo
visits =  7405
orders =  68
promo 7405 68 0.009182984469952735
vk 256
source =  vk
visits =  256
orders =  2
vk 256 2 0.0078125
yandex 11757
source =  yandex
visits =  11757
orders =  104
yandex 11757 104 0.008845793995066768
direct 2156
source =  direct
visits =  215

In [1]:
import pandas as pd

In [72]:
names_visits = ['name', 'visits']

In [73]:
data = pd.read_csv('visits_by_source.csv', sep = ';', header=None, names = names_visits)
data.head()

,name,visits
0,burgerclub,1197
1,city-magazine,528
2,facebook,3144
3,food-delivery,1184
4,foody,421


# Шаг 2. Запись результатов в файл

### Запись результатов в файл

Т. к. файл visits_by_source.txt может быть очень большим, выводить результаты на экран не особо практично.

Давайте запишем результат в файл.

Для этого мы будем «открывать» файл joined_by_source.txt с параметром 'w' вместо 'r'. При каждом таком «открытии» файл joined_by_source.txt будет создаваться заново (т. е., если он существовал ранее, все его содержимое будет удалено).

Один файл (visits_by_source.txt) у нас уже открыт как переменная f. Соответственно, при открытии файла joined_by_source.txt надо давать ему другое имя (например, f_joined). И вместо функции print пишем команду записи в файл f_joined.write(...).

Обратите внимание, что при записи строки в файл надо указывать символ переноса строки \n в конце каждой строчки, иначе все строки будут записаны как одна:

In [32]:
with open('joined_by_source.txt', 'w') as f_joined:
    with open('visits_by_source.txt', 'r') as f:
        for line in f:
            line = line.strip().split('\t')
            source = line[0]
            visits = int(line[1])
            orders = searchForLine( source, orders_dict )
            f_joined.write('{}\t{}\t{}\n'.format( source, visits, orders) )

### Упражнение

Добавьте в файл orders_by_source.txt первой строчкой заголовок, чтобы после работы скрипта результаты можно было, например, скопировать в Excel в готовом к использованию виде.

Теперь добавьте расчет конверсии визитов в покупки четвертым столбцом.

Какая средняя конверсия для всех источников (т. е. отношение всех покупок к сумме визитов)? Ответ округлите до тысячных (т. е. ответ будет иметь формат 0.009).

In [36]:
names_orders = ['source', 'p1', 'orders']

In [37]:
data = pd.read_csv('orders_by_source.csv', sep = ';', header=None, names = names_orders)
data.head()

,source,p1,orders
0,burgerclub,10,185
1,city-magazine,5,81
2,direct,5,88
3,facebook,5,91
4,food-delivery,10,173


In [40]:
with open('home_dz.csv', 'w') as fj:
    with open('orders_by_source.txt', 'r') as f:
        for line in f:
            line = line.strip().split('\t')
            source = line[0]
            orders = int(line[1])
            visits = searchForLine( source, visits_dict )
            conversion = orders / visits
            fj.write('{}\t{}\t{}\t{}\n'.format( source, visits, orders, conversion) )

In [41]:
names_orders_j = ['source', 'visits', 'orders', 'conversion']

In [45]:
data = pd.read_csv('home_dz.csv', sep = '\t', header=None, names = names_orders_j)
data.head(11)

,source,visits,orders,conversion
0,burgerclub,1197,10,0.008354
1,city-magazine,528,5,0.009470
2,direct,2156,5,0.002319
3,facebook,3144,5,0.001590
4,food-delivery,1184,10,0.008446
5,foody,421,3,0.007126
6,google,10961,77,0.007025
7,newsletter,637,5,0.007849
8,promo,7405,68,0.009183
9,vk,256,2,0.007812


In [61]:
data.sum(axis=0)

source        burgerclubcity-magazinedirectfacebookfood-deli...
visits                                                    39646
orders                                                      294
conversion                                            0.0780207
dtype: object

In [65]:
data.visits.sum()

39646

In [66]:
data.orders.sum()

294

In [67]:
data.orders.sum()/data.visits.sum()

0.007415628310548353

# Блок 3. Объединение таблиц по двум измерениям > Шаг 1. Построение вложенного словаря

В этом блоке нам понадобятся данные о визитах и покупках, сгруппированные по источнику и каналу.

Данные по визитам (visits_by_source_and_medium.txt)

Данные по покупкам (orders_by_source_and_medium.txt)

Поиск по одному столбцу — ситуация не самая часто встречающаяся. Обычно столбцов гораздо больше. Да и файлы, с которыми приходится работать, на практике гораздо сложнее. Как «масштабировать» наше решение, чтобы оно могло учитывать любое количество столбцов и при этом не теряло скорость?

Для удобства представим более реальную таблицу с покупками. Например, на 4 миллиона строк и 500 Мб на диске. Если необходимо найти в этой таблице нужную комбинацию source и medium, надо учитывать, что эта пара может встретиться в любой из 4 миллионов строк. Также может быть ситуация, что для этой комбинации source и medium не было ни одной покупки.

Для ускорения нашего поиска давайте заменим таблицу из столбцов source, medium и количества покупок вложенным словарем. Пример такого преобразования:

In [69]:
# Соответствующий словарь:
orders_dict = {
    'google': {
        'sem': 56,
        'seo': 15
    },    
    'newsletter': {
        'email': 5
    }
}

Т. е. первичными ключами словаря будут все значения переменной source. Для каждого ключа словаря в качестве значений создаем еще один словарь, ключами которого уже будут значения столбца medium, и в качестве значений ставим количество покупок orders.

### Что дает такая структура?

Теперь, чтобы найти нужную комбинацию значений source и medium, нужно сделать два простых шага:

1. Из 2000 значений source определить, есть ли среди них нужный нам. Если его не оказалось, то сразу возвращаем 0 покупок.

2. Если значение source есть в первичных ключах, то ищем значение medium в списке из вторичных ключей, которые оказались у источника source (кстати, их может быть всего несколько). Логика такая же: если ключ найден, возвращаем значение покупок для первичного ключа source и вторичного medium. Если не найден, то возвращаем 0.

Итого для поиска среди 4 000 000 строк нам нужно сначала проверить наличие очередного значения в списке ключей из  source, а затем — в списке из medium. Это намного быстрее, чем любой последовательный мониторинг этих 4 миллионов строк.

Давайте реализуем этот алгоритм в коде. Сначала преобразуем знакомый нам словарь orders_dict, чтобы из таблицы со столбцами source и medium получать словарь с двумя уровнями ключей.

### Метод setdefault

Для более наглядной записи используем метод setdefault, который позволяет не проводить проверку наличия ключа в словаре. Чтобы прибавить к значению словаря число мы использовали следующую проверку:

In [23]:
orders_dict = {}
if 'google' in orders_dict:
    orders_dict['google'] += 1
else:
    orders_dict['google'] = 1
print(orders_dict)

{'google': 1}


Можно переписать этот код с помощью метода setdefault. Этот метод проверяет есть ли в словаре указанный ключ 'google'. Если есть, то оставляет соответствующее значение ключа прежним. Если ключа не оказалось, то подставляет указанное нами значение (в примере это значение 0). Тем самым после применения метода setdefault можно смело использовать прибавление 1 к ключу 'google'. Независимо от того был ли этот ключ в словаре раньше:

In [25]:
orders_dict = {}
orders_dict.setdefault('google', 0)
orders_dict['google'] += 1
print(orders_dict)

{'google': 1}


Давайте сформируем словарь orders_dict из прошлого блока. Только теперь в нем добавится еще один уровень вложенности для канала medium:

In [27]:
orders_dict = {}
with open('orders_by_source_and_medium.txt', 'r') as f:
    for line in f:
        line = line.strip().split('\t')       
        source = line[0]
        medium = line[1]
        orders_count = int( line[2] )      
        orders_dict.setdefault(source, {})
        orders_dict[source].setdefault(medium, 0)      
        orders_dict[source][medium] = orders_count

In [28]:
print(orders_dict)

{'burgerclub': {'cpa-partners': 10}, 'city-magazine': {'cpc-partners': 5}, 'direct': {'brand': 5}, 'facebook': {'smm': 5}, 'food-delivery': {'cpa-partners': 10}, 'foody': {'cpc-partners': 3}, 'google': {'brand': 6, 'sem': 56, 'seo': 15}, 'newsletter': {'email': 5}, 'promo': {'email': 68}, 'vk': {'smm': 2}, 'yandex': {'brand': 7, 'sem': 67, 'seo': 30}}


Смотрим что получилось для ключа 'google':

In [30]:
orders_dict['yandex']

{'brand': 7, 'sem': 67, 'seo': 30}

### Домашнее задание

Измените код функции searchForLine из прошлого блока, чтобы по названию источника source и канала medium получать соответствующее количество покупок из файла orders_by_source_and_medium.txt. Соответственно, функция должна "пройти" следующие тесты:

In [35]:
def searchForLine_DZ( source, medium, dict_s ):
    
    #print(dict_s)
    if source in dict_s:
        print (source)
        if medium in dict_s[source]:
            return int( dict_s[source][medium] )
        else:
            return (0)
    else:
        return (0)

In [36]:
searchForLine_DZ('google', 'seo', orders_dict)

google


15

In [37]:
searchForLine_DZ('google_123', 'seo', orders_dict)

0

In [38]:
searchForLine_DZ('google', 'seo_123', orders_dict)

google


0

In [39]:
searchForLine_DZ('google', 'sem', orders_dict)

google


56

# Шаг 2. Тест на повторение пройденных материалов

In [43]:
a = 4 / 2
type(a)

float

In [44]:
months = ['январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь']

In [53]:
# Как получить из этого листа названия весенних месяцев?
months[2:5]

['март', 'апрель', 'май']

In [47]:
campaign = 'yandex_cpc_bmv_brand_summer'

In [52]:
# Как выделить из переменной campaign название марки автомобиля?
campaign.split('_')[2]

'bmv'

In [50]:
 campaign.split('_')[2:4]

['bmv', 'brand']

In [51]:
total_costs = { 'google': 1319, 'yandex': 1818, 'promo': 1181 }

In [54]:
# Необходимо вывести названия источников, отсортированных по алфавиту в обратном порядке
sorted( total_costs, reverse = True )

['yandex', 'promo', 'google']

In [55]:
sorted( total_costs.values(), reverse = True )

[1818, 1319, 1181]

In [56]:
sorted( total_costs.keys(), reverse = True )

['yandex', 'promo', 'google']

In [59]:
def google_costs(data):

    data.setdefault('google', 0)

    return data['google']

In [60]:
google_costs( {'vk': 100} )

0

In [62]:
# datetime: 
date_string = '23.02.2017 15:20'

In [67]:
datetime.strptime( date_string, '%d.%m.%Y %H:%M' )

AttributeError: 'str' object has no attribute 'strptime'

# Дополнительный блок. Объединение таблиц по произвольным столбцам > Введение

### ПОНЯТИЕ РЕКУРСИИ

Осторожно, рекурсия! Этот блок посвящен очень сложной теме объединения таблиц с 5-ю и более столбцами с помощью рекурсии. Мы вынесли его и готовое решение в дополнительную информацию для самых любознательных.

Вы наверняка заметили, что в прошлом шаге при учете столбца medium наш код усложнился. Приходилось каждый раз проверять наличие новых ключей или использовать setdefault для всех уровней. В реальных задачах столбцов будет больше. Например, в некоторых задачах приходится объединять таблицы по 5 и более столбцам. Если обходиться проверками или методом setdefault на всех возможных уровнях словарей, то наш код будет быстро разрастаться или станет медленным, т. к. заранее сложно предсказать структуру вложенных уровней словаря orders_dict.

В таких задачах на помощь приходит использование рекурсии. Это понятие широко используется в науке и большом количестве задач.

В текущем модуле под рекурсией мы будем понимать алгоритм, в котором функция может вызывать сама себя с изменяющимся аргументом.

В текущем модуле под рекурсией мы будем понимать алгоритм, в котором функция может вызывать сама себя с изменяющимся аргументом.

Рассмотрим математический пример: допустим, нам необходимо вывести все числа от 10 до 1 с шагом 1. Если не использовать функцию range, то можно сделать это следующим образом:

In [68]:
i = 10
while i >= 1:
    print(i)
   
    # уменьшаем значение i на 1
    # такая запись равносильна i = i - 1
    i -=1

10
9
8
7
6
5
4
3
2
1


В цикле мы на каждом шаге уменьшаем значение переменной i и ждем, пока ее значение не снизится до 1. В этом примере перебор крайне простой, т. к. в качестве шага алгоритма выступает уменьшение переменной на 1. Но что делать, если изменение переменной алгоритма сложно выразить в цикле? Например, если нам нужно перебрать ключи словаря, вложенные друг в друга? Не составлять же нам многократно вручную цепочки вида orders_dict[source][medium][date]...

Давайте решим пример с переменной i следующим образом: вместо цикла while и строчки i -= 1 используем функцию:

In [69]:
def decrease_and_print_i( i ):
    if i > 1:
        print(i)
        return decrease_and_print_i(i - 1)
    else:
        return 1

Это пример рекурсивной функции. Рассмотрим как она работает. При первом вызове этой функции со значением i = 10 идет проверка: если i > 1, то вывести i на экран и вызвать эту функцию со значением i = 9.

При вызове функции со значением i = 9 снова происходит проверка i > 1, i выводится на экран и вызывается функция со значением i = 8.

Так происходит до тех пор, пока значение i не станет меньше 1. Тогда функция возвращает 1 и завершает работу.

### Упражнение

Часто в качестве примера рекурсии приводят расчет факториала числа. Т. е. для числа n это выражение n!:

Как написать рекурсивную функцию, которая рассчитывает факториал числа?

В основе функции можно использовать более короткую запись определения факториала:

In [70]:
n! = n * (n - 1)!
1! = 1

SyntaxError: invalid syntax (<ipython-input-70-cd1fb7aad6be>, line 1)

In [85]:
def factor_i( i, U ):
    if i > 1:
        print('i', i)
        print('U*i', U*i)
        U = U*i
        return factor_i(i - 1, U)
    else:
        return U

In [86]:
F = 1
print('F', factor_i( 4, F ))

i 4
U*i 4
i 3
U*i 12
i 2
U*i 24
F 24


### Рекурсия и словари

Все прошлые примеры можно было без труда решить с помощью циклов. Давайте сделаем пример, в котором использование рекурсии позволит нам перевести лист в словарь

In [87]:
line = ['2016-10-01', 'burgerclub', 'cpa-partners', 1]

In [88]:
dict2fill = {'2016-10-01': {'burgerclub': {'cpa-partners': 1}}}

Здесь в общем случае простым циклом уже не обойтись, т. к. нам надо создать словарь с большим количеством вложенных ключей.

Наша функция должна проходить по всем значениям листа и для каждого создавать новый ключ внутри «внешнего» словаря. Чтобы реализовать такой алгоритм можем использовать рекурсию. На каждом шаге берем очередной элемент листа и создаем пару ключ-значение. В качестве ключа ставим текущий элемент листа. А в качестве значения берем вызов этой же функции, но с урезанным на 1 элемент листом.

In [92]:
line = ['2016-10-01', 'burgerclub', 'cpa-partners', 1]
def fillLevels( lineRemainder ):
    """
fillLevels( ['2016-10-01', 'burgerclub', 'cpa-partners', 1] )
    {'2016-10-01': {'burgerclub': {'cpa-partners': 1}}}
    """
    
    # словарь, который будем пошагово заполнять
    dict2fill = {}
    if len( lineRemainder ) > 1:
        dict2fill[ lineRemainder[0] ] = fillLevels( lineRemainder[1:] )
    else:
        return lineRemainder[0]
    return dict2fill

На вход функция принимает часть листа line. 

    Берет его первый элемент lineRemainder[0] и вызывает себя с остатком листа line, т. е. с элементами lineRemainder[1:].

    Так продолжаем до тех пор, пока в lineRemainder не останется один элемент

    Пример

Что получилось:

In [94]:
fillLevels(line)

{'2016-10-01': {'burgerclub': {'cpa-partners': 1}}}

### Упражнение

Получите вложенный словарь для листа range(100).

Какое значение будет иметь ключ в самого глубоко вложенного словаря?

In [96]:
fillLevels(range(100))

{0: {1: {2: {3: {4: {5: {6: {7: {8: {9: {10: {11: {12: {13: {14: {15: {16: {17: {18: {19: {20: {21: {22: {23: {24: {25: {26: {27: {28: {29: {30: {31: {32: {33: {34: {35: {36: {37: {38: {39: {40: {41: {42: {43: {44: {45: {46: {47: {48: {49: {50: {51: {52: {53: {54: {55: {56: {57: {58: {59: {60: {61: {62: {63: {64: {65: {66: {67: {68: {69: {70: {71: {72: {73: {74: {75: {76: {77: {78: {79: {80: {81: {82: {83: {84: {85: {86: {87: {88: {89: {90: {91: {92: {93: {94: {95: {96: {97: {98: 99}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}

# Построение алгоритма поиска по нескольким столбцам
Теперь, используя перевод листа в словарь через функцию fillLevels, мы можем дополнять получившийся словарь другими строками. Т. е. мы хотим из списка листов

data = [
    ['2016-10-01', 'google', 'sem', 5],
    ['2016-10-01', 'google', 'seo', 1],
    ['2016-10-01', 'newsletter', 'email', 1]
]

получить словарь

data_dict = 
    { '2016-10-01': 
        { 'google': 
            { 'sem': 5, 
              'seo': 1 },      
          'newsletter': 
            { 'email': 1 }
        }
    }

### Функция перевода таблицы в словарь

Здесь нам на помощь опять придет рекурсия:

1. На первом шаге словарь data_dict пуст. Поэтому с помощью функции fillLevels просто записываем в него первую строку ['2016-10-01', 'google', 'sem', 5] в виде словаря.

2. При обработке второй строки line = ['2016-10-01', 'google', 'seo', 1] нужно проверить, нет ли в словаре data_dict уже существующих ключей. Такую проверку можно сделать рекурсивно, последовательно сравнивая ключи словаря data_dict и элементы листа line. Как только мы найдем несовпадение, дополним словарь data_dict остатком листа line с помощью функции fillLevels

In [99]:
def checkLevels( levelDict, level, line ):
    if line[ level ] in levelDict:
        checkLevels( levelDict[ line[ level ] ], level + 1, line )
        return levelDict
    else:
        levelDict[ line[ level ] ] =  fillLevels( line[ level + 1: ] )
        return levelDict

Проверим, что получится:

In [100]:
data = [
    ['2016-10-01', 'google', 'sem', 5],
    ['2016-10-01', 'google', 'seo', 1],
    ['2016-10-01', 'newsletter', 'email', 1]
]
data_dict = {}
for line in data:
    data_dict = checkLevels(data_dict, 0, line)
print(data_dict)

{'2016-10-01': {'google': {'sem': 5, 'seo': 1}, 'newsletter': {'email': 1}}}


### Упражнение

Статистика покупок прошедшей рекламной кампании задается шестью параметрами листом data:

In [101]:
data = [

['2018-01-01', 'google', 'cpc', 'ДФО', 'кампания_1', 'Хабаровск', 114],

['2018-01-01', 'google', 'cpc', 'ДФО', 'кампания_2', 'Владивосток', 536],

['2018-01-01', 'google', 'cpc', 'ДФО', 'кампания_1', 'Магадан', 436]

]

Используя функцию checkLevels, получите словарь data_dict.

Сколько различных кампаний было проведено в листе data? Т. е. необходимо посчитать количество ключей словаря data_dict['2018-01-01']['google']['cpc']['ДФО']

In [102]:
data_dict = {}
for line in data:
    data_dict = checkLevels(data_dict, 0, line)
print(data_dict)

{'2018-01-01': {'google': {'cpc': {'ДФО': {'кампания_1': {'Хабаровск': 114, 'Магадан': 436}, 'кампания_2': {'Владивосток': 536}}}}}}


In [103]:
data_dict['2018-01-01']['google']['cpc']['ДФО']

{'кампания_1': {'Хабаровск': 114, 'Магадан': 436},
 'кампания_2': {'Владивосток': 536}}

### Поиск покупок в таблице

Словарь готов, а это значит, основная часть задачи выполнена. Осталось написать функцию поиска количества покупок по трем значениям: дата покупки, источник и канал.

Реализуем следующий алгорим: будем последовательно проверять наличие ключа с данной датой, источником или каналом в словаре data_dict. Если хотя бы одного ключа не окажется, то можем сразу возвращать 0 покупок. Это существенно ускорит наш поиск в случае присутствия больших таблиц.

In [104]:
def findLineValue( finalDict, line ):
    if len( line ) > 1:
        if line[ 0 ] in finalDict:
            return findLineValue( finalDict[ line[ 0 ] ], line[1:] )
        else:
            return 0
    else:
        if line[0] in finalDict:
            return finalDict[ line[0] ]
        else:
            return 0

In [108]:
# Проверим корректность работы:

findLineValue( data_dict, ['2018-01-01', 'google', 'cpc'] )

{'ДФО': {'кампания_1': {'Хабаровск': 114, 'Магадан': 436},
  'кампания_2': {'Владивосток': 536}}}

In [107]:
# Если даты нет в словаре data_dict, то должны получить 0:

findLineValue( data_dict, ['2018-01-01', 'google', 'sem'] )

0

### Упражнение

Известно, что 1 января по кампании google / cpc кампания_2 во Владивостоке было 26 800 переходов. Посчитайте конверсию из переходов в покупки, используя функцию поиска findLineValue.

Каково значение конверсии переходов в покупки для этого случая? Приведите точное значение конверсии (пример формата ответа 0.04)

In [126]:
a = findLineValue( data_dict, ['2018-01-01', 'google', 'cpc', 'ДФО', 'кампания_2'] )
#type (a)
print(a)
print(a.items())
print(a.keys())
print(a.values())

{'Владивосток': 536}
dict_items([('Владивосток', 536)])
dict_keys(['Владивосток'])
dict_values([536])


In [120]:
a.values()

dict_values([536])

In [123]:
536/26800

0.02